In [38]:
type Direction = | Up | Down | Left | Right

type Move = 
    { 
        Direction : Direction
        Steps : int 
    }

type Rope = 
    {
        Head : int*int
        Tail : int*int
        TailVisited : (int*int) list
    }

module Rope =
    let starting = { Head = (0,0); Tail = (0,0); TailVisited = [(0,0)] }

    let moveHead dir rope =
        let (x,y) = rope.Head
        match dir with
        | Up -> { rope with Head = (x, y + 1) }
        | Down -> { rope with Head = (x, y - 1) }
        | Left -> { rope with Head = (x - 1, y) }
        | Right -> { rope with Head = (x + 1, y) }        

    let moveTail rope =
        let (hx, hy) = rope.Head
        let (tx, ty) = rope.Tail
        let (dx, dy) = (hx-tx, hy-ty)
        if (abs dx <= 1) && (abs dy <= 1) then
            rope // ends are next to each other
        else
            let t' = (tx + sign dx, ty + sign dy) // move one space max
            { rope with Tail = t'; TailVisited = t'::rope.TailVisited }

    let rec applyMove move rope =
        if move.Steps = 0 then
            rope
        else
            rope
            |> moveHead move.Direction
            |> moveTail
            |> applyMove { move with Steps = move.Steps-1 }

    let applyMoves moves rope =
        moves
        |> Seq.fold (fun r m -> applyMove m r) rope

    let visitedCount rope =
        rope.TailVisited
        |> List.distinct
        |> List.length


In [39]:
let parseLine (s : string) =
    let steps = int (s.Substring(2))
    let dir = 
        match s[0] with
        | 'U' -> Up
        | 'D' -> Down
        | 'L' -> Left
        | 'R' -> Right
        | _ -> failwith "Unrecognised direction"
    { Direction = dir; Steps = steps }


In [40]:
#r "nuget: FsUnit"

open FsUnitTyped

let testInput = 
    [|
        "R 4"
        "U 4"
        "L 3"
        "D 1"
        "R 4"
        "D 1"
        "L 5"
        "R 2"
    |]
    |> Array.map parseLine

let testResult = Rope.starting |> Rope.applyMoves testInput

Rope.visitedCount testResult |> shouldEqual 13

Installed Packages FsUnit, 5.1.0

In [41]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_09.txt")
let moves = 
    File.ReadAllLines(sourcePath)
    |> Array.map parseLine

let result = 
    Rope.starting
    |> Rope.applyMoves moves
    |> Rope.visitedCount

In [ ]:
printfn "Tail visited %d spaces" result

## Part 2

In [49]:
type Knot = 
    { 
        Position : int*int
        History : (int*int) list
    }

type Rope2 = 
    {
        Knots : Knot list
    }

module Rope2 =
    let starting numKnots = 
        {
            Knots = List.init numKnots (fun _ -> { Position = (0,0); History = [(0,0)] })
        }

    let moveTail (ahead, behind) =
        let (hx, hy) = ahead.Position
        let (tx, ty) = behind.Position
        let (dx, dy) = (hx-tx, hy-ty)
        if (abs dx <= 1) && (abs dy <= 1) then
            behind // ends are next to each other
        else
            let t' = (tx + sign dx, ty + sign dy) // move one space max
            { Position = t'; History = t' :: behind.History }

    let pull dir rope =
        match rope.Knots with
        | [] -> rope
        | h::rest ->
            let (x,y) = h.Position
            let h' =
                match dir with
                | Up -> (x, y + 1)
                | Down -> (x, y - 1)
                | Left -> (x - 1, y)
                | Right -> (x + 1, y)
            
            let movedHead =
                { Position = h'; History = h'::h.History }
            
            let movedTail =
                movedHead::rest
                |> List.pairwise
                |> List.map moveTail

            { rope with Knots = movedHead::movedTail }

    let rec applyMove move rope =
        if move.Steps = 0 then
            rope
        else
            rope
            |> pull move.Direction
            |> applyMove { move with Steps = move.Steps-1 }

    let applyMoves moves rope =
        moves
        |> Seq.fold (fun r m -> applyMove m r) rope

    let lastVisitedCount rope =
        (List.last rope.Knots).History
        |> List.distinct
        |> List.length



In [54]:
let largeTest =
    [|
        "R 5"
        "U 8"
        "L 8"
        "D 3"
        "R 17"
        "D 10"
        "L 25"
        "U 20"
    |]
    |> Array.map parseLine

let largeTestResult = Rope2.starting 10 |> Rope2.applyMoves largeTest

//Rope2.lastVisitedCount largeTestResult |> shouldEqual 36

In [55]:
largeTestResult

Knots


In [50]:
let result = 
    Rope2.starting 10
    |> Rope2.applyMoves moves
    |> Rope2.lastVisitedCount

In [51]:
printfn "Tail visited %d spaces" result

Tail visited 2687 spaces
